### -------------------------------------  Closed-ended  SQL Queries ---------------------------

### Q1. What are the top 5 brands by receipts scanned among users 21 and over?

In [ ]:

--- Calculates users' ages based on BIRTH_DATE.
WITH UserAge AS (
    SELECT 
        column0 AS user_id,  -- ID from USER_TAKEHOME
        TRY_CAST(SUBSTRING(column2, 1, 10) AS DATE) AS birth_date,  -- Extract YYYY-MM-DD
        EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM TRY_CAST(SUBSTRING(column2, 1, 10) AS DATE)) AS age
    FROM USER_TAKEHOME
    WHERE column2 IS NOT NULL
),
-- Filters out users 21 and over
EligibleUsers AS (
    SELECT user_id FROM UserAge WHERE age >= 21
)
SELECT 
    p.BRAND AS brand,  
    COUNT(DISTINCT t.RECEIPT_ID) AS receipt_count  
FROM TRANSACTION_TAKEHOME t
JOIN EligibleUsers u ON t.user_id = u.user_id  
JOIN PRODUCTS_TAKEHOME p ON t.BARCODE = p.BARCODE
GROUP BY p.BRAND
ORDER BY receipt_count DESC
LIMIT 6;

--- NOTE:  Results had NULL at top with most receipts scanned so to exclude that added the 6th brand

### RESULT 

In [ ]:
Rank	Brand	Receipts Scanned
0	       	                  3
0	DOVE	                  3
1	NERDS CANDY	              3
2	SOUR PATCH KIDS	          2
3	COCA-COLA	              2
4	TRIDENT	                  2

### Q2. What are the top 5 brands by sales among users that have had their account for at least six months?

In [ ]:
-- Get DayActive of Users
WITH UserAccountAge AS (
    SELECT 
        column0 AS user_id,  -- ID from USER_TAKEHOME
        TRY_CAST(SUBSTRING(column1, 1, 10) AS DATE) AS created_date,  -- Extract YYYY-MM-DD
        DATE_DIFF('day', TRY_CAST(SUBSTRING(column1, 1, 10) AS DATE), CURRENT_DATE) AS days_active
    FROM USER_TAKEHOME
    WHERE column1 IS NOT NULL
),
-- Filter for users active atleast for 6 months
EligibleUsers AS (
    SELECT user_id 
    FROM UserAccountAge 
    WHERE days_active >= 180  -- At least 6 months
),

-- TOP 5 brand sales
BrandSales AS (
    SELECT 
        p.brand AS brand,  
        SUM(
            CASE 
                WHEN TRIM(t.FINAL_SALE) = '' THEN 0  -- Replace empty strings with 0
                WHEN TRY_CAST(t.FINAL_SALE AS DECIMAL) IS NULL THEN 0  -- Handle non-numeric values
                ELSE TRY_CAST(t.FINAL_SALE AS DECIMAL)  
            END
        ) AS total_sales
    FROM TRANSACTION_TAKEHOME t
    JOIN EligibleUsers u ON t.user_id = u.user_id  
    JOIN PRODUCTS_TAKEHOME p ON t.BARCODE = p.BARCODE  
    GROUP BY p.brand
)
SELECT brand, total_sales
FROM BrandSales
ORDER BY total_sales DESC
LIMIT 6;

--- NOTE:  Results had NULL in betweeen with most total sales so to exclude that added the 6th brand

In [ ]:
Rank	Brand	Total Sales
0	CVS	              72000
1	TRIDENT	          46720
2	DOVE  	          42880
3	COORS LIGHT	      34960
4	        	      16650
5	Quaker 	          16600